In [ ]:
!nvcc --version
%pip install nvcc4jupyter
%load_ext nvcc4jupyter

In [8]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

#define n 20
#define BLOCK_SIZE 256

__global__ 
void addArrays(float *a, float *b, float *c) {
    __shared__ float sharedA[BLOCK_SIZE];
    __shared__ float sharedB[BLOCK_SIZE];

    int index = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (index < n) {
        sharedA[threadIdx.x] = a[index];
        sharedB[threadIdx.x] = b[index];
    }
    __syncthreads();

    if (index < n) {
        c[index] = sharedA[threadIdx.x] + sharedB[threadIdx.x];
    }
}


int main() {
    float a[n], b[n], c[n];
    float *d_a, *d_b, *d_c;

    size_t sz = n * sizeof(float);
    
    for (int i = 0; i < n; ++i) {
        a[i] = i;
        b[i] = 2 * i;
    }
    
    cudaMalloc((void **)&d_a, sz);
    cudaMalloc((void **)&d_b, sz);
    cudaMalloc((void **)&d_c, sz);
    
    cudaMemcpy(d_a, a, sz, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sz, cudaMemcpyHostToDevice);

    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;

    addArrays<<<numBlocks, blockSize>>>(d_a, d_b, d_c);

    cudaMemcpy(c, d_c,sz, cudaMemcpyDeviceToHost);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    
    for (int i = 0; i < n; ++i) {
        printf("%5.3f + %5.3f = %5.3f\n", a[i], b[i], c[i]);
    }

    return 0;
}


0.000 + 0.000 = 0.000
1.000 + 2.000 = 3.000
2.000 + 4.000 = 6.000
3.000 + 6.000 = 9.000
4.000 + 8.000 = 12.000
5.000 + 10.000 = 15.000
6.000 + 12.000 = 18.000
7.000 + 14.000 = 21.000
8.000 + 16.000 = 24.000
9.000 + 18.000 = 27.000
10.000 + 20.000 = 30.000
11.000 + 22.000 = 33.000
12.000 + 24.000 = 36.000
13.000 + 26.000 = 39.000
14.000 + 28.000 = 42.000
15.000 + 30.000 = 45.000
16.000 + 32.000 = 48.000
17.000 + 34.000 = 51.000
18.000 + 36.000 = 54.000
19.000 + 38.000 = 57.000

